In [ ]:
'''
Questo modello esegue un primo fine-tuning
Il modello verrà poi salvato, ed utilizzato in seguito per addestrare il modello finale (Final)
'''

In [ ]:
# Caricamento libreirie
from fastai.vision.all import *
from fastai.torch_core import set_seed

import os
import glob
import pandas as pd
from torch.nn import CrossEntropyLoss

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
# Settaggio iper-parametri e generatore random
SEED = 123
IMG_SIZE = 512
BS = 8

set_seed(SEED, reproducible=True)

In [ ]:
# Creazione dataframe
masks = sorted(glob.glob('/kaggle/input/debris/PythonImagesMasks/mask_*.png'))

df = pd.DataFrame({'mask':masks})
df['image'] = df['mask'].apply(lambda x: x.replace('mask', 'image'))


bad_imgs = []

for f in df['image'].to_list():
    try:
        Image.open(f)
    except:
        bad_imgs.append(f)

df = df.loc[~df['image'].isin(bad_imgs)]

In [ ]:
# Rimozione validation-set (dopo aver eseguito il training del modello) 
df['is_valid'] = False

In [ ]:
# Definizione dataloader
codes = ['Background', 'Debris']

dblock = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW), MaskBlock(codes)), 
    get_x=ColReader('image'),
    get_y=ColReader('mask'),
    #splitter=ColSplitter(col='is_valid'),
    splitter=RandomSplitter(valid_pct=0.2, seed=SEED),
    #item_tfms=Resize(IMG_SIZE),
    batch_tfms=[Flip()],
)

dls = dblock.dataloaders(df, bs=BS)
dls.c = dls.train.after_item.c

In [ ]:
#dblock.summary(df)
#dls.one_batch()

#print('Mini-batch shape: ', list(dls.one_batch()[0].shape))
#print('Targets shape', list(dls.one_batch()[1].shape))

In [ ]:
# Visualizzazione dataset
dls.show_batch(cmap='jet', vmin=0, vmax=3)

In [ ]:
# Inizializzazione pesi loss-function
wgts = torch.tensor([1, 62]).float().cuda() #62

In [ ]:
# Definizione architettura e ulteriori iper-parametri per il training 
learn = unet_learner(
    dls, resnet18, #efficientnet_b0
    n_in=1,
    n_out=dls.c,
    loss_func=CrossEntropyLoss(weight=wgts),
    opt_func=ranger,
    normalize=False,
    self_attention=True,
).to_fp16()

In [ ]:
# Sblocco pesi modello per fine-tuning
#learn.unfreeze()

In [ ]:
# Tool stimatore del learning rate
#learn.lr_find()

In [ ]:
# Training del modello
lr = 5e-4

learn.fit_flat_cos(2, slice(lr)) #2 epoche

In [ ]:
# Salvataggio del modello

#learn.save('self_att_unet', with_opt=False)

learn.export('self_att_unet.pkl')

In [ ]:
# Visualizzaione dei risultati ottenuti sul validation-set
learn.show_results(cmap='gray')

In [ ]:
# Inferenza e risultati su test-set
imgs_test = glob.glob('/kaggle/input/debris/Images_casted/*.png')
msks_test = glob.glob('/kaggle/input/debris/casted_masks_1px/*.png')

df_test = pd.DataFrame({'image':imgs_test, 'mask':msks_test})

test_dl = dls.test_dl(df_test, with_labels=True, shuffle=False)

preds = learn.get_preds(dl=test_dl)

res = learn.validate(dl=test_dl)
print('Risultati su Test-set: ', round(res[0],3))

In [ ]:
final = torch.argmax(preds[0][:16], 1)
img = Image.fromarray(final[13].numpy().astype('uint8') * 255)

img